In [1]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
import numpy as np
import os

import time
from sys import platform

from models import *
from utils.datasets import *
from utils.utils import *
from torch.utils.data import DataLoader
import shutil 
from IPython.display import clear_output
from collections import defaultdict

In [ ]:
# input: a list of all trained image paths, a list of all available image paths 
# output: append full image path of the next image that should be annotated to new_metadata
#         a distances_to_trained dictionary storing {img path:np array of distances to all imgs in trained_imgs}
def sampleNextImage (distances_to_trained,
                     trained_specimens=defaultdict(int),
                     trained_images=[],
                     all_image_paths=[],
                     class_names = {},
                     trained_image_metadata= "./metadata/truck_data/medAL_sampling_all.txt", 
                     all_image_metadata = "./metadata/truck_data/vehicle_all_random.txt", 
                     new_metadata = "./metadata/truck_data/medAL_sampling_all.txt"):
    if trained_image_metadata != new_metadata:
        shutil.copy(trained_image_metadata, new_metadata)
    if distances_to_trained == {}:
        first_time = True
        
    # parse txt files
    def read_paths(metadata):
        image_paths = []
        with open(metadata,"r") as f:
            image_paths += f.readlines()

        # Strip all the newlines
        image_paths = [p.rstrip() for p in image_paths]
        return image_paths
    
    # if first time, build new trained_labels, images, specimens 
    if distances_to_trained == {}:
        trained_image_paths = read_paths(trained_image_metadata)
        all_image_paths = read_paths(all_image_metadata)
        # make a list of trained images and trained label files as np arrays
#         trained_specimens = defaultdict(int) # {specimen:count of this specimen in trained data}
        for p in trained_image_paths:
            im = cv2.imread(p).flatten() # shape = (416*416*3,)
            trained_images.append(im)
            label_file = p.replace("images/","labels/").replace(".png",".txt")
            if os.path.exists(label_file):
                with open(label_file,"r") as file:
                    labels = np.loadtxt(file)
                    if labels.ndim <= 1: 
                        labels = np.array([labels])
                    labels.tolist()
                    for l in labels: 
                        class_name, x, y, w, h = l
                        trained_specimens[class_name] += 1
        
    # compute distances 
    untrained_images = []
    max_avg_dist = -1
    max_avg_dist_unique_specimen = -1
    
    next_image_path = None
    next_image_class_names = None
    next_image_label_file = None
    
    for p in all_image_paths: 
        if p not in trained_image_paths:
            # modify dictionary so that every value is subtracted by min . that way we can pick the specimen with least images
            min_occurance = min(trained_specimens.values())
            for spec in trained_specimens:
                trained_specimens[spec] -= min_occurance
            
            im = cv2.imread(p).flatten()
            untrained_images.append(im)
            if p not in distances_to_trained:  
                distances = np.sum(np.square(im - trained_images), axis = 1) # actually is distance squared, length = # trained img
                distances_to_trained[p] = distances
            else:
                assert(len(distances_to_trained[p]) == len(trained_images)-1)
                distance_to_last_sampled = np.sum(np.square(im - trained_images[-1])) # len = 0
                distances_to_trained[p]=np.append(distances_to_trained[p],distance_to_last_sampled)
                distances = distances_to_trained[p]
            
            # find next best image
            # find the image that has a class label that was least seen AND with max avg distance 
            contains_rarest_class = False
            if first_time:
                    label_file = p.replace("images/","labels/").replace(".png",".txt")
                    current_image_class_names = []
                    if os.path.exists(label_file):
                        with open(label_file,"r") as file:
                            labels = np.loadtxt(file)
                            if labels.ndim <= 1: 
                                labels = np.array([labels])
                            labels.tolist()
                            for l in labels: 
                                class_name, x, y, w, h = l
                                current_image_class_names += class_name
                                if class_name not in trained_specimens or trained_specimens[class_name] == 0:
                                    contains_rarest_class = True
                    class_names[p] = current_image_class_names 
            else: 
                current_image_class_names = class_names[p]
                for class_name in current_image_class_names:
                    if class_name not in trained_specimens or trained_specimens[class_name] == 0:
                        contains_rarest_class = True
            avg_dist = np.mean(distances)
            
            if avg_dist > max_avg_dist_unique_specimen and contains_rarest_class:
                next_image_path = p 
                max_avg_dist_unique_specimen = avg_dist 
            if avg_dist > max_avg_dist:
                max_avg_dist = avg_dist 
                if max_avg_dist_unique_specimen == -1:
                    next_image_path = p
            if next_image_path == p:
                next_image_class_names = current_image_class_names 
                next_image_label_file = label_file
                next_im = im
                
                
    # write next_image_path to new_metadata
    with open(new_metadata,"a") as f: 
        f.write("\n" + next_image_path)
    
    im = cv2.imread(next_image_path).flatten() # shape = (416*416*3,)
    
    # update outputs 
    trained_image_paths.append(next_image_path)
    if len(next_image_class_names)!=0:
        for class_name in next_image_class_names:
            trained_specimens[class_name] += 1 
    distances_to_trained.pop(next_image_path, None)
    
    print("Wrote " + next_image_path + " to " + new_metadata)
    print("which now has " + str(len(trained_images) + 1) + " images")

    return distances_to_trained, trained_specimens, trained_images, all_image_paths, class_names

In [ ]:
# one image at a time
distances_to_trained, trained_specimens, trained_images, all_image_paths, class_names=sampleNextImage({})
print(distances_to_trained)

In [ ]:
len(distances_to_trained)

In [ ]:
# all images
for i in range(3499):
    distances_to_trained, trained_specimens, trained_images, all_image_paths, class_names = sampleNextImage(distances_to_trained, trained_specimens, trained_images, all_image_paths, class_names)

# Distance only

In [4]:
def calculateDistance(i1, i2):
    return np.sum((i1-i2)**2)
def read_paths(metadata):
        image_paths = []
        with open(metadata,"r") as f:
            image_paths += f.readlines()

        # Strip all the newlines
        image_paths = [p.rstrip() for p in image_paths]
        return image_paths


In [16]:
  def compute_distances_one_loop(X,X_train):

    """    
    Inputs:
    - X: A numpy array of shape (num_test, D) containing test data.
    Returns:
    - dists: A numpy array of shape (num_test, num_train) where dists[i, j]
      is the Euclidean distance between the ith test point and the jth training
      point.
    """
    num_test = X.shape[0]
    num_train = X_train.shape[0]
    dists = np.zeros((num_test, num_train)) 
    dists = np.sqrt(np.sum(X**2, axis=1).reshape(num_test, 1) + np.sum(X_train**2, axis=1) - 2 * X.dot(X_train.T))

    #########################################################################
    return dists

In [17]:
train_image_paths = read_paths("./metadata/truck_data/vehicle_train_medAL.txt")
all_image_paths = read_paths("./metadata/truck_data/vehicle_train_random.txt")
test_image_paths= [x for x in all_image_paths if x not in train_image_paths]
dists= None
print(len(test_image_paths))

train_images = []
test_images = []
for p in test_image_paths:
    im = cv2.imread(p)
    im=cv2.resize(im,(128,128)).flatten()
    test_images.append(im)
for p in train_image_paths:
    im = cv2.imread(p)
    im=cv2.resize(im,(128,128)).flatten()
    train_images.append(im)


2599


In [18]:
f=open(train_image_paths,"a");
first_time = True
while(len(test_image_paths)!=0):
    if first_time:
        dists = compute_distances_one_loop(np.array(test_images), np.array(train_images))
        first_time=False
    else: 
        dists_last_column = compute_distances_one_loop(np.array(test_images),np.array(train_images[-1]).reshape(1,-1))
        dists = dists[:-1,:] # remove last row 
        dists= np.c_[dists, dists_last_column]
    print(dists.shape)
    avg_dists = np.mean(dists,axis=1)
    ind = np.argmin(avg_dists)

    f.write("\n" + test_image_paths[ind])
    train_image_paths.append(test_image_paths.pop(ind))
    train_images.append(test_images.pop(ind))
f.close()

(2599, 1)
(2598, 2)
(2597, 3)
(2596, 4)
(2595, 5)
(2594, 6)
(2593, 7)
(2592, 8)
(2591, 9)
(2590, 10)
(2589, 11)
(2588, 12)
(2587, 13)
(2586, 14)
(2585, 15)
(2584, 16)
(2583, 17)
(2582, 18)
(2581, 19)
(2580, 20)
(2579, 21)
(2578, 22)
(2577, 23)
(2576, 24)
(2575, 25)
(2574, 26)
(2573, 27)
(2572, 28)
(2571, 29)
(2570, 30)
(2569, 31)
(2568, 32)
(2567, 33)
(2566, 34)
(2565, 35)
(2564, 36)
(2563, 37)
(2562, 38)
(2561, 39)
(2560, 40)
(2559, 41)
(2558, 42)
(2557, 43)
(2556, 44)
(2555, 45)
(2554, 46)
(2553, 47)
(2552, 48)
(2551, 49)
(2550, 50)
(2549, 51)
(2548, 52)
(2547, 53)
(2546, 54)
(2545, 55)
(2544, 56)
(2543, 57)
(2542, 58)
(2541, 59)
(2540, 60)
(2539, 61)
(2538, 62)
(2537, 63)
(2536, 64)
(2535, 65)
(2534, 66)
(2533, 67)
(2532, 68)
(2531, 69)
(2530, 70)
(2529, 71)
(2528, 72)
(2527, 73)
(2526, 74)
(2525, 75)
(2524, 76)
(2523, 77)
(2522, 78)
(2521, 79)
(2520, 80)
(2519, 81)
(2518, 82)
(2517, 83)
(2516, 84)
(2515, 85)
(2514, 86)
(2513, 87)
(2512, 88)
(2511, 89)
(2510, 90)
(2509, 91)
(2508, 9

(1907, 693)
(1906, 694)
(1905, 695)
(1904, 696)
(1903, 697)
(1902, 698)
(1901, 699)
(1900, 700)
(1899, 701)
(1898, 702)
(1897, 703)
(1896, 704)
(1895, 705)
(1894, 706)
(1893, 707)
(1892, 708)
(1891, 709)
(1890, 710)
(1889, 711)
(1888, 712)
(1887, 713)
(1886, 714)
(1885, 715)
(1884, 716)
(1883, 717)
(1882, 718)
(1881, 719)
(1880, 720)
(1879, 721)
(1878, 722)
(1877, 723)
(1876, 724)
(1875, 725)
(1874, 726)
(1873, 727)
(1872, 728)
(1871, 729)
(1870, 730)
(1869, 731)
(1868, 732)
(1867, 733)
(1866, 734)
(1865, 735)
(1864, 736)
(1863, 737)
(1862, 738)
(1861, 739)
(1860, 740)
(1859, 741)
(1858, 742)
(1857, 743)
(1856, 744)
(1855, 745)
(1854, 746)
(1853, 747)
(1852, 748)
(1851, 749)
(1850, 750)
(1849, 751)
(1848, 752)
(1847, 753)
(1846, 754)
(1845, 755)
(1844, 756)
(1843, 757)
(1842, 758)
(1841, 759)
(1840, 760)
(1839, 761)
(1838, 762)
(1837, 763)
(1836, 764)
(1835, 765)
(1834, 766)
(1833, 767)
(1832, 768)
(1831, 769)
(1830, 770)
(1829, 771)
(1828, 772)
(1827, 773)
(1826, 774)
(1825, 775)
(182

(1253, 1347)
(1252, 1348)
(1251, 1349)
(1250, 1350)
(1249, 1351)
(1248, 1352)
(1247, 1353)
(1246, 1354)
(1245, 1355)
(1244, 1356)
(1243, 1357)
(1242, 1358)
(1241, 1359)
(1240, 1360)
(1239, 1361)
(1238, 1362)
(1237, 1363)
(1236, 1364)
(1235, 1365)
(1234, 1366)
(1233, 1367)
(1232, 1368)
(1231, 1369)
(1230, 1370)
(1229, 1371)
(1228, 1372)
(1227, 1373)
(1226, 1374)
(1225, 1375)
(1224, 1376)
(1223, 1377)
(1222, 1378)
(1221, 1379)
(1220, 1380)
(1219, 1381)
(1218, 1382)
(1217, 1383)
(1216, 1384)
(1215, 1385)
(1214, 1386)
(1213, 1387)
(1212, 1388)
(1211, 1389)
(1210, 1390)
(1209, 1391)
(1208, 1392)
(1207, 1393)
(1206, 1394)
(1205, 1395)
(1204, 1396)
(1203, 1397)
(1202, 1398)
(1201, 1399)
(1200, 1400)
(1199, 1401)
(1198, 1402)
(1197, 1403)
(1196, 1404)
(1195, 1405)
(1194, 1406)
(1193, 1407)
(1192, 1408)
(1191, 1409)
(1190, 1410)
(1189, 1411)
(1188, 1412)
(1187, 1413)
(1186, 1414)
(1185, 1415)
(1184, 1416)
(1183, 1417)
(1182, 1418)
(1181, 1419)
(1180, 1420)
(1179, 1421)
(1178, 1422)
(1177, 1423)

(590, 2010)
(589, 2011)
(588, 2012)
(587, 2013)
(586, 2014)
(585, 2015)
(584, 2016)
(583, 2017)
(582, 2018)
(581, 2019)
(580, 2020)
(579, 2021)
(578, 2022)
(577, 2023)
(576, 2024)
(575, 2025)
(574, 2026)
(573, 2027)
(572, 2028)
(571, 2029)
(570, 2030)
(569, 2031)
(568, 2032)
(567, 2033)
(566, 2034)
(565, 2035)
(564, 2036)
(563, 2037)
(562, 2038)
(561, 2039)
(560, 2040)
(559, 2041)
(558, 2042)
(557, 2043)
(556, 2044)
(555, 2045)
(554, 2046)
(553, 2047)
(552, 2048)
(551, 2049)
(550, 2050)
(549, 2051)
(548, 2052)
(547, 2053)
(546, 2054)
(545, 2055)
(544, 2056)
(543, 2057)
(542, 2058)
(541, 2059)
(540, 2060)
(539, 2061)
(538, 2062)
(537, 2063)
(536, 2064)
(535, 2065)
(534, 2066)
(533, 2067)
(532, 2068)
(531, 2069)
(530, 2070)
(529, 2071)
(528, 2072)
(527, 2073)
(526, 2074)
(525, 2075)
(524, 2076)
(523, 2077)
(522, 2078)
(521, 2079)
(520, 2080)
(519, 2081)
(518, 2082)
(517, 2083)
(516, 2084)
(515, 2085)
(514, 2086)
(513, 2087)
(512, 2088)
(511, 2089)
(510, 2090)
(509, 2091)
(508, 2092)
(507

In [19]:
f.close()

In [95]:
print(dists.shape)

(3165, 317)


In [10]:
dists_last_column = compute_distances_one_loop(np.array(test_images),np.array(train_images[-1]).reshape(1,-1))


IndexError: list index out of range